# Heat Exchanger 1D Unit Model with Ideal & IAPWS Property Package

![](heat_exchanger_4.svg)

**Problem Statement**: In this example, we will be heating a benzene-toluene mixture using steam. 

**Tube Side Inlet**

Flow Rate = 250 mol/s

Mole fraction (Benzene) = 0.4

Mole fraction (Toluene) = 0.6

Pressure = 101325 Pa

Temperature = 350 K

**Shell Side Inlet**

Flow Rate = 100 mol/s

Mole fraction (Steam) = 1

Pressure = 101325 Pa

Temperature = 450 K

This example will demonstrate the simulation of the 1D heat exchanger by fixing any 7 of the following degrees of freedom:
- two of shell length, diameter, and area
- two of tube length, diameter, and area
- number of tubes
- wall temperature (at all spatial points)
- heat transfer coefficient (at all spatial points, for both shell and tube)


IDAES documentation reference for heat exchanger 1D model: https://idaes-pse.readthedocs.io/en/latest/technical_specs/model_libraries/generic/unit_models/heat_exchanger_1D.html

**Setting up the problem in IDAES**

In [1]:
# Import pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, value, units

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

#import the BTX property package to create a properties block for the flowsheet
from idaes.generic_models.properties.activity_coeff_models import BTX_activity_coeff_VLE

# Import the IAPWS property package to create a properties block for the flowsheet
from idaes.generic_models.properties import iapws95

from idaes.generic_models.properties.iapws95 import htpx

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.generic_models.properties.core.examples.BT_ideal \
    import configuration

#Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom

#Import a heat exchanger unit
from idaes.generic_models.unit_models.heat_exchanger_1D import (HeatExchanger1D,
                                                                HeatExchangerFlowPattern,
                                                                WallConductionType)

#Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

# Steady State Model
m.fs = FlowsheetBlock(default={"dynamic": False})

# Setup property packages for shell and tube side
# Steam property package
m.fs.properties_shell = iapws95.Iapws95ParameterBlock()

# BT ideal property package
m.fs.properties_tube = GenericParameterBlock(default=configuration)

2022-03-03 13:46:45 [WARNING] idaes.generic_models.properties.core.pure.Perrys: DEPRECATED - fs.properties_tube.benzene dens_mol_liq_comp_coeff index 'eqn_type' should be specified, defaulting to equation form 1.
2022-03-03 13:46:45 [WARNING] idaes.generic_models.properties.core.pure.Perrys: DEPRECATED - fs.properties_tube.toluene dens_mol_liq_comp_coeff index 'eqn_type' should be specified, defaulting to equation form 1.


In the [0D Heat Exchanger model example](http://localhost:8888/notebooks/GitHub/examples-pse/src/Examples/UnitModels/Operations/heat_exchanger_0D_testing.ipynb), geometry effects are ignored in favor of temperature gradients to calculate heat transfer. Here, we need to specify discretization, a flow configuration and wall conduction assumption. We specify the one-dimensional spatial discretization to use backwards finite difference approximations with 20 finite elements - these are the defaults if none are specified explicitly, and the shell and tube domains may be discretized differently if desired. The domains must use the same number of finite elements, as the exchanger is linear and the elements directly correspond according to the selected flow pattern.

The 1D Heat Exchanger supports the following options:

- Flow Configuration
    - HeatExchangerFlowPattern.cocurrent. Shell and tube flow in parallel, tube inlet transfers with shell inlet and similar with outlets, and temperature difference is greatest at the flow inlets (default).
    - HeatExchangerFlowPattern.countercurrent. Sshell and tube flow in anti-parallel, tube inlet transfers with shell outlet and vice versa, and temperature difference changes minimally along the exchanger length.
- Wall Conduction Assumption
    - WallConductionType.zero_dimensional. 0D wall model (default).
    - WallConductionType.one_dimensional. 1D wall model along the thickness of the tube (not currently supported in IDAES)
    - WallConductionType.two_dimensional. 2D wall model along the length and thickness of the tube (not currently supported in IDAES)

In [2]:
#Create an instance of the heat exchanger unit, attaching it to the flowsheet
#Specify that the property package to be used with the heater is the one we created earlier.
m.fs.heat_exchanger = HeatExchanger1D(default={
        "shell_side":{"property_package": m.fs.properties_shell,
                      "transformation_method": "dae.finite_difference",
                      "transformation_scheme": "BACKWARD"},
        "tube_side":{"property_package": m.fs.properties_tube,
                     "transformation_method": "dae.finite_difference",
                     "transformation_scheme": "BACKWARD"},
        "finite_elements": 20,
        "flow_type": HeatExchangerFlowPattern.cocurrent,
        "has_wall_conduction": WallConductionType.zero_dimensional})

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))



The initial DOF is 56


In [3]:
assert DOF_initial == 56

In [4]:
h = htpx(450*units.K, P = 101325*units.Pa)

#Fix the inlet conditions
m.fs.heat_exchanger.shell_inlet.flow_mol.fix(100) #mol/s
m.fs.heat_exchanger.shell_inlet.pressure.fix(101325)
m.fs.heat_exchanger.shell_inlet.enth_mol.fix(h) #J/mol

DOF_initial = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_initial))


The DOF is 53


In [5]:
m.fs.heat_exchanger.tube_inlet.flow_mol.fix(250)# mol/s
m.fs.heat_exchanger.tube_inlet.mole_frac_comp[0, "benzene"].fix(0.4)
m.fs.heat_exchanger.tube_inlet.mole_frac_comp[0, "toluene"].fix(0.6)
m.fs.heat_exchanger.tube_inlet.pressure.fix(101325) #Pa
m.fs.heat_exchanger.tube_inlet.temperature[0].fix(350)#K

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

The DOF is 48


### Option 1: Fix HTC and dimensions of each domain


In [6]:
m.fs.heat_exchanger.d_shell.fix(1.04) #m
m.fs.heat_exchanger.d_tube_outer.fix(0.01167) #m
m.fs.heat_exchanger.d_tube_inner.fix(0.01067) #m
m.fs.heat_exchanger.N_tubes.fix(10) #dimensionless
m.fs.heat_exchanger.shell_length.fix(4.85) #m
m.fs.heat_exchanger.tube_length.fix(4.85) #m
m.fs.heat_exchanger.shell_heat_transfer_coefficient.fix(2000) #W/m2/K
m.fs.heat_exchanger.tube_heat_transfer_coefficient.fix(51000) #W/m2/K

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

The DOF is 0


In [7]:
assert DOF_final == 0

In [8]:
#Initialize the flowsheet, and set the output at WARNING
m.fs.heat_exchanger.initialize()

#Solve the simulation using ipopt
#Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
solve_status = opt.solve(m, tee = True)

#Display a readable report
m.fs.heat_exchanger.report()

2022-03-03 13:46:46 [INFO] idaes.init.fs.heat_exchanger.shell: Initialization Complete
2022-03-03 13:46:46 [INFO] idaes.init.fs.heat_exchanger.tube.properties: Starting initialization
2022-03-03 13:46:47 [INFO] idaes.init.fs.heat_exchanger.tube.properties: Dew and bubble point initialization: optimal - Optimal Solution Found.
2022-03-03 13:46:47 [INFO] idaes.init.fs.heat_exchanger.tube.properties: Equilibrium temperature initialization completed.
2022-03-03 13:46:47 [INFO] idaes.init.fs.heat_exchanger.tube.properties: Phase equilibrium initialization: optimal - Optimal Solution Found.
2022-03-03 13:46:47 [INFO] idaes.init.fs.heat_exchanger.tube.properties: Property initialization: optimal - Optimal Solution Found.
2022-03-03 13:46:47 [INFO] idaes.init.fs.heat_exchanger.tube: Initialization Complete
2022-03-03 13:46:49 [INFO] idaes.init.fs.heat_exchanger: Initialization Complete.
Ipopt 3.13.2: 

******************************************************************************
This program 

In [9]:
from pyomo.opt import TerminationCondition, SolverStatus
import pytest

# Check if termination condition is optimal
assert solve_status.solver.termination_condition == TerminationCondition.optimal
assert solve_status.solver.status == SolverStatus.ok

assert value(m.fs.heat_exchanger.shell.properties[0.0, 1.0].temperature) == pytest.approx(400.60, abs=1e-2)
assert value(m.fs.heat_exchanger.tube_outlet.temperature[0]) == pytest.approx(368.39, abs=1e-2)


### Option 2: Unfix shell length  and fix shell outlet temperatures

In the previous example, we fixed the heat exchanger area and overall heat transfer coefficient. However, given that the models in IDAES are equation oriented, we can fix the outlet variables. For example, we can fix the shell outlet temperature and heat exchanger diameter, and solve for the heat exchanger shell length that will satisfy that condition.


In [10]:
m.fs.heat_exchanger.shell_length.unfix()
m.fs.heat_exchanger.shell_outlet.enth_mol.fix(htpx(400*units.K, P = 101325*units.Pa))
print(degrees_of_freedom(m))

0


In [11]:
result = opt.solve(m)

print(result)

#Display a readable report
m.fs.heat_exchanger.report()


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 929
  Number of variables: 929
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Solved To Acceptable Level.
  Termination condition: optimal
  Id: 1
  Error rc: 0
  Time: 0.44794678688049316
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Unit : fs.heat_exchanger                                                   Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                 : Value      : Fixed : Bounds
        Number of Tubes :     10.000 :  True : (None, None)
             Shell Area :    0.84842 : False : (None, None)
         Shell Diameter :     1.0400 :  True : (None, None)
           Shell Length :     4.9515 : False : (None, None)
              Tube Area : 8.9417e-05 : False : (None, None)
    Tube Inner Diameter :   0.010670 :  True : (None, 

In [12]:
# Check if termination condition is optimal
assert solve_status.solver.termination_condition == TerminationCondition.optimal
assert solve_status.solver.status == SolverStatus.ok

assert value(m.fs.heat_exchanger.shell_length) == pytest.approx(4.95, abs=1e-2)
assert value(m.fs.heat_exchanger.tube_outlet.temperature[0]) == pytest.approx(368.39, abs=1e-2)
